In [1]:
!pip install geopy

In [2]:
# Imports
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut
import time

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.options.display.float_format = '{:.2f}'.format

In [3]:
apt = pd.read_csv('apt_SP_cleaned.csv')

In [6]:
apt.head()

,registration_number,postal_code,transaction_value,transaction_date,built_area
0,5.101800e+09,03144-080,500000,2021-12-21,65
1,1.831200e+10,05782-470,413000,2021-12-22,63
2,1.240911e+10,05171-600,380000,2021-12-22,178
3,1.092000e+10,02355-000,1050000,2021-12-21,357
4,5.914001e+09,03662-020,345000,2021-12-23,60


In [5]:
apt.drop(columns = 'neighborhood', inplace = True)

In [7]:
geolocator = GoogleV3(api_key="API_KEY")

In [8]:
# Função para obter coordenadas e bairro a partir do CEP
def get_coordinates_from_cep(cep):
    try:
        location = geolocator.geocode(cep)
        if location:
            # Tenta extrair o bairro dos dados do endereço
            neighborhood = None
            for component in location.raw.get("address_components", []):
                if "sublocality" in component["types"]:
                    neighborhood = component["long_name"]
                    break
            return location.latitude, location.longitude, neighborhood
        else:
            return None, None, None
    except GeocoderTimedOut:
        print(f"Erro de timeout ao buscar dados para o CEP {cep}. Tentando novamente...")
        return get_coordinates_from_cep(cep)  # Tenta novamente em caso de timeout
    except Exception as e:
        print(f"Erro ao buscar dados para o CEP {cep}: {e}")
        return None, None, None

In [9]:
# Função para aplicar a geocodificação, exibir progresso e salvar o CSV a cada 50 requisições
def geocode_with_progress(row, index, total_rows, apt):
    latitude, longitude, neighborhood = get_coordinates_from_cep(row['postal_code'])
    if (index + 1) % 50 == 0:  # Exibe mensagem a cada 50 requisições
        print(f"Requisições concluídas: {index + 1} de {total_rows}")
        # Salva o dataframe em .csv a cada 50 requisições
        apt.to_csv(f'apt_progress_{index + 1}.csv', index=False)
        print(f"Arquivo salvo como 'apt_progress_{index + 1}.csv'")
    time.sleep(1)  # Atraso de 1 segundo entre requisições para evitar sobrecarga
    return latitude, longitude, neighborhood

In [11]:
# Aplicando a função para preencher as colunas de latitude, longitude e bairro
for index, row in apt.iterrows():
    latitude, longitude, neighborhood = geocode_with_progress(row, index, len(apt), apt)
    apt.at[index, 'latitude'] = latitude
    apt.at[index, 'longitude'] = longitude
    apt.at[index, 'neighborhood'] = neighborhood

Requisições concluídas: 50 de 6316
Arquivo salvo como 'apt_progress_50.csv'
Requisições concluídas: 100 de 6316
Arquivo salvo como 'apt_progress_100.csv'
Requisições concluídas: 150 de 6316
Arquivo salvo como 'apt_progress_150.csv'
Requisições concluídas: 200 de 6316
Arquivo salvo como 'apt_progress_200.csv'
Requisições concluídas: 250 de 6316
Arquivo salvo como 'apt_progress_250.csv'
Requisições concluídas: 300 de 6316
Arquivo salvo como 'apt_progress_300.csv'
Requisições concluídas: 350 de 6316
Arquivo salvo como 'apt_progress_350.csv'
Requisições concluídas: 400 de 6316
Arquivo salvo como 'apt_progress_400.csv'
Requisições concluídas: 450 de 6316
Arquivo salvo como 'apt_progress_450.csv'
Requisições concluídas: 500 de 6316
Arquivo salvo como 'apt_progress_500.csv'
Requisições concluídas: 550 de 6316
Arquivo salvo como 'apt_progress_550.csv'
Requisições concluídas: 600 de 6316
Arquivo salvo como 'apt_progress_600.csv'
Requisições concluídas: 650 de 6316
Arquivo salvo como 'apt_progre

Requisições concluídas: 5200 de 6316
Arquivo salvo como 'apt_progress_5200.csv'
Requisições concluídas: 5250 de 6316
Arquivo salvo como 'apt_progress_5250.csv'
Requisições concluídas: 5300 de 6316
Arquivo salvo como 'apt_progress_5300.csv'
Requisições concluídas: 5350 de 6316
Arquivo salvo como 'apt_progress_5350.csv'
Requisições concluídas: 5400 de 6316
Arquivo salvo como 'apt_progress_5400.csv'
Requisições concluídas: 5450 de 6316
Arquivo salvo como 'apt_progress_5450.csv'
Requisições concluídas: 5500 de 6316
Arquivo salvo como 'apt_progress_5500.csv'
Requisições concluídas: 5550 de 6316
Arquivo salvo como 'apt_progress_5550.csv'
Requisições concluídas: 5600 de 6316
Arquivo salvo como 'apt_progress_5600.csv'
Requisições concluídas: 5650 de 6316
Arquivo salvo como 'apt_progress_5650.csv'
Requisições concluídas: 5700 de 6316
Arquivo salvo como 'apt_progress_5700.csv'
Requisições concluídas: 5750 de 6316
Arquivo salvo como 'apt_progress_5750.csv'
Requisições concluídas: 5800 de 6316
Arq

In [17]:
apt.isna().sum()

registration_number      0
postal_code              0
transaction_value        0
transaction_date         0
built_area               0
latitude               120
longitude              120
neighborhood           146
dtype: int64

In [18]:
apt = apt.dropna()

In [19]:
apt.isna().sum()

registration_number    0
postal_code            0
transaction_value      0
transaction_date       0
built_area             0
latitude               0
longitude              0
neighborhood           0
dtype: int64

In [20]:
display(apt)

,registration_number,postal_code,transaction_value,transaction_date,built_area,latitude,longitude,neighborhood
0,5.101800e+09,03144-080,500000,2021-12-21,65,-23.596041,-46.574510,Vila Lucia
1,1.831200e+10,05782-470,413000,2021-12-22,63,-23.650386,-46.765836,Parque Munhoz
2,1.240911e+10,05171-600,380000,2021-12-22,178,-23.473614,-46.750166,Conjunto Residencial Vista Verde
3,1.092000e+10,02355-000,1050000,2021-12-21,357,-23.456164,-46.612475,Jardim Virginia Bianca
4,5.914001e+09,03662-020,345000,2021-12-23,60,-23.522384,-46.497763,Vila Ré
...,...,...,...,...,...,...,...,...
6311,1.220990e+10,05847-500,660000,2023-07-12,208,-23.652428,-46.748946,Jardim Iracema
6312,1.701940e+10,05713-570,690000,2023-07-21,214,-23.624770,-46.729349,Jardim Ampliacao
6313,1.134270e+10,03687-170,375000,2023-07-24,68,-23.525760,-46.486938,Jardim Artur Alvim
6314,9.515300e+09,04786-035,1500000,2023-07-31,308,-23.702521,-46.705402,Interlagos


In [22]:
apt.to_csv('apt_SP_pointsadded.csv', index = False)